In [1]:
# Import everything that will be used:
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import cv2

from getABSDData import getABSDDataMask # We will use this function for the training data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# The width of the image:
img_width = 224
# The heigh of the image:
img_height = 224  

# We use the VGG19 model for image classification with weights trained on ImageNet:
model = keras.applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
model.layers.pop()
model.layers.pop()

# Adding custom Layers 
x = model.layers[-1].output
x = Flatten()(x)
predictions = Dense(img_width*img_height, activation="relu")(x)

# Creating the final model 
model_final = Model(input = model.input, output = predictions)

# Compile the model, we use mean squared error for the loss function, and SGD algorithm for optimization:
model_final.compile(loss = "mean_squared_error", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [3]:
# We can get some information about our final net:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
# Resize the data using the actual img_width and img_width:
resizer = lambda x:cv2.resize(x, (img_width, img_width))

# We can receive the training data using the getABSDDataMask function:
training,_,_ = getABSDDataMask(1, label_converter=resizer, image_converter=resizer)

In [ ]:
# The number of the epochs:
n_iterations = 10

# Learning:
model_final.fit_generator(training, steps_per_epoch=1, epochs=n_iterations, verbose=2)

Epoch 1/10
